In [1]:
##### Module G6 Challenge
  ### I. Get the Weather Description and Amount of Precipitation for Each City
   ## I.2.a Add the latitudes and longitudes to a list.
   ## I.3.Get the nearest city using the citipy module. 
   ## I.4-5.Perform an API Call with the OpenWeatherMap.
    #    Return: Latitude and Longitude, Max Temp, Percent Humidity, Percent Cloudiness, Wind Speed, Weather description  
    #    Return: Rainfall -use try-except, if it is raining get rainfall # in last three hours. If not raining, add 0 for city
    #    Return: Snow -use try-except, if it is snowing get snowfall # in last three hours. If not snowing, add 0 for city    
   ## I.6. Add collected data to new dataframe
   ## I.7. Save new DataFrame as CSV to be used later in Part II.  weatherPy_challenge.csv
   ## I.8. Git hub 
   ## I.9. ?? Using Pandas: How many cities have recorded rainfall or snowfall


In [2]:
###   Prepare Environment   
import requests  # needed for web crawls 
import pandas as pd # our Panda friend
import matplotlib.pyplot as plt # matplotlib for plotting
import numpy as np  # numpy for calculations
from citipy import citipy # Wendy Peng's amazing cities library
from datetime import datetime # working with dates
import time  # working with time

##  import our api keys
from config import weather_api_key  ## for weather data
from config import g_key ## for gmaps heatmaps

In [150]:
###  I.2. Generate a new set of 1,500 random latitude and longitude combinations.
#         Using NumPy random number generation with dictated lows, high and size of sample
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)  ## 
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs) ## packs both variables into a tupled zip file
lat_lngs # show me the zip file

In [151]:
##  I.2.a Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs) #write the zip file into a list, so you can work with it

In [152]:
###  I.3.Get the nearest city using the citipy module. 

## Create a list for holding the cities.
cities = []  # empty list

## Identify the nearest city for each latitude and longitude combination.
## The citipy module finds the nearest city to the latitude and longitude pair with a population of 500 or more.

for coordinate in coordinates:  # loop thru coordinates llist of 1500 lats and lngs
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    try:
    # If the city is unique, then we will add it to the cities list.
        if city not in cities:
            cities.append(city)
    except():
        print("General Error... skipping.")   
        # Print the city count to confirm sufficient count.
len(cities)

600

In [6]:
### I.4-5.Perform an API Call with the OpenWeatherMap.
 
 ## I.4.a General a url for querying openweathermap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

# We will need to do the following:

# Import our dependencies and initialize counters and an empty list that will hold the weather data.
   
# Loop through the cities list.
# Group the cities in sets of 50 to log the process as we find the weather data for each city.
# Two counters will be needed here: one to log the city count from 1 to 50, and another for the sets.

# Build the city_url or endpoint for each city.
# Log the URL and the record and set numbers.
# Make an API request for each city.
# Parse the JSON weather data for the following:
# City, country, and date
# Latitude and longitude
# Maximum temperature
# Humidity
# Cloudiness
# Wind speed
# Add the data to a list in a dictionary format and then convert the list to a DataFrame
#    Return: Latitude and Longitude, Max Temp, Percent Humidity, Percent Cloudiness, Wind Speed, Weather description  
#    Return: Rainfall -use try-except, if it is raining get rainfall # in last three hours. If not raining, add 0 for city
#    Return: Snow -use try-except, if it is snowing get snowfall # in last three hours. If not snowing, add 0 for city    





http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=058ccb02f765c0268667a6f8eeac8cb8


In [155]:
### I.5 Perform an API Call with the OpenWeatherMap.

##  Prepare empty city list for processing the import

# start with an empty list for weather data
city_data=[]  # empty list

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1  # number of records in crawl
set_count = 1  # # of sets in crawl.  50 cities per set.

# Loop through all the cities in the list.
for i, city in enumerate(cities): # for loop with enumerate. reference index and city var

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50): ## if index divisble by 50 and index> 50, creates a new group
        set_count += 1  # adds one to set count 
        record_count = 1 # resets record count to 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        
        ## Challenge I.5 Weather Description
        city_weather_main=city_weather["weather"][0]["main"]   
        city_weather_desc = city_weather["weather"][0]["description"]
        
        ## Challenge I.5 try - Get inches of rain in last three hours where it has rained.
        try:
            if city_weather["weather"][0]["main"]=="Rain":
                city_rain_inches= city_weather["rain"]["3h"]
            else: 
                city_rain_inches=0
        except:  print (f"failing on the rain. skipping")
             
        ## Challenge I.5 try - except practice.  Now do with snow
        try:
            if city_weather["weather"][0]["main"]=="Snow":
                city_snow_inches= city_weather["snow"]["3h"]
            else: 
                city_snow_inches=0
        except:  print (f"failing on the snow. skipping")
              
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Overall Weather" : city_weather_main,
                          "Weather Description" : city_weather_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Rain Inches (last 3 hrs)" : city_rain_inches,
                          "Snow Inches (last 3 hrs)" : city_snow_inches
                          })
         
                         
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass  ## general purpose statement to handle all errors and continue

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | arraial do cabo
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | skagastrond
City not found. Skipping...
Processing Record 4 of Set 1 | hermanus
Processing Record 5 of Set 1 | agirish
Processing Record 6 of Set 1 | samarai
Processing Record 7 of Set 1 | biak
Processing Record 8 of Set 1 | barawe
City not found. Skipping...
Processing Record 9 of Set 1 | sistranda
Processing Record 10 of Set 1 | nanortalik
Processing Record 11 of Set 1 | atuona
Processing Record 12 of Set 1 | albany
Processing Record 13 of Set 1 | muyezerskiy
Processing Record 14 of Set 1 | kapaa
Processing Record 15 of Set 1 | hargeysa
Processing Record 16 of Set 1 | barrow
failing on the snow. skipping
Processing Record 17 of Set 1 | myrhorod
Processing Record 18 of Set 1 | rawson
Processing Record 19 of Set 1 | sampit
Processing Record 20 of Set 1 | port elizabeth
Processing Record 21 of Set 1 | manta
Proce

Processing Record 30 of Set 4 | marsa matruh
Processing Record 31 of Set 4 | inuvik
Processing Record 32 of Set 4 | mamboma
Processing Record 33 of Set 4 | coquimbo
Processing Record 34 of Set 4 | cherskiy
Processing Record 35 of Set 4 | callaway
Processing Record 36 of Set 4 | evanston
Processing Record 37 of Set 4 | rab
City not found. Skipping...
Processing Record 38 of Set 4 | warqla
City not found. Skipping...
Processing Record 39 of Set 4 | bandar-e lengeh
Processing Record 40 of Set 4 | illapel
Processing Record 41 of Set 4 | san patricio
Processing Record 42 of Set 4 | calamar
Processing Record 43 of Set 4 | bandarbeyla
Processing Record 44 of Set 4 | hasaki
Processing Record 45 of Set 4 | wewak
Processing Record 46 of Set 4 | mys shmidta
City not found. Skipping...
Processing Record 47 of Set 4 | barranca
Processing Record 48 of Set 4 | buala
Processing Record 49 of Set 4 | parabel
Processing Record 50 of Set 4 | hithadhoo
Processing Record 1 of Set 5 | boundiali
Processing Re

Processing Record 10 of Set 8 | trinidad
Processing Record 11 of Set 8 | sitka
Processing Record 12 of Set 8 | maghama
City not found. Skipping...
Processing Record 13 of Set 8 | deniliquin
Processing Record 14 of Set 8 | birnin kebbi
Processing Record 15 of Set 8 | nanakuli
Processing Record 16 of Set 8 | guaymas
Processing Record 17 of Set 8 | berbera
Processing Record 18 of Set 8 | leirvik
Processing Record 19 of Set 8 | kieta
Processing Record 20 of Set 8 | tazovskiy
Processing Record 21 of Set 8 | port macquarie
Processing Record 22 of Set 8 | okha
Processing Record 23 of Set 8 | ust-nera
Processing Record 24 of Set 8 | ambilobe
Processing Record 25 of Set 8 | moerai
Processing Record 26 of Set 8 | sisimiut
Processing Record 27 of Set 8 | touros
Processing Record 28 of Set 8 | chara
Processing Record 29 of Set 8 | okhotsk
Processing Record 30 of Set 8 | nantucket
Processing Record 31 of Set 8 | nizhneyansk
City not found. Skipping...
Processing Record 32 of Set 8 | acari
Processin

Processing Record 41 of Set 11 | fairbanks
failing on the snow. skipping
Processing Record 42 of Set 11 | alenquer
Processing Record 43 of Set 11 | marawi
Processing Record 44 of Set 11 | poso
Processing Record 45 of Set 11 | saint-leu
Processing Record 46 of Set 11 | chitose
Processing Record 47 of Set 11 | terre-de-bas
Processing Record 48 of Set 11 | atarfe
Processing Record 49 of Set 11 | ruatoria
City not found. Skipping...
Processing Record 50 of Set 11 | pyaozerskiy
Processing Record 1 of Set 12 | plettenberg bay
Processing Record 2 of Set 12 | acapulco
Processing Record 3 of Set 12 | port hedland
Processing Record 4 of Set 12 | shimanovsk
Processing Record 5 of Set 12 | baykit
Processing Record 6 of Set 12 | haibara
Processing Record 7 of Set 12 | kashary
Processing Record 8 of Set 12 | puerto leguizamo
Processing Record 9 of Set 12 | ye
City not found. Skipping...
Processing Record 10 of Set 12 | kudahuvadhoo
Processing Record 11 of Set 12 | ituni
City not found. Skipping...
P

In [156]:
### I.6 Add the collected data to a new dataframe.
city_data_df=pd.DataFrame(city_data)
city_data_df.head(5)
#city_data

,City,Country,Date,Lat,Lng,Overall Weather,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Arraial Do Cabo,BR,2020-01-14 02:29:54,-22.97,-42.03,Clouds,overcast clouds,76.28,74,100,7.34,0.0,0.0
1,Mataura,NZ,2020-01-14 02:36:58,-46.19,168.86,Clouds,overcast clouds,62.01,66,98,10.00,0.0,0.0
2,Hermanus,ZA,2020-01-14 02:29:55,-34.42,19.23,Clouds,overcast clouds,72.00,87,86,8.99,0.0,0.0
3,Agirish,RU,2020-01-14 02:29:55,61.92,63.02,Clouds,overcast clouds,15.51,94,93,3.31,0.0,0.0
4,Samarai,PG,2020-01-14 02:29:55,-10.62,150.67,Clouds,overcast clouds,86.00,67,100,10.80,0.0,0.0


In [158]:
### I.7  Output the content to a CSV file so we have static data to work with...
output_data_file = "weather_data/weatherPy_challenge.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
   ## I.6. Add collected data to new dataframe
   ## I.7. Save new DataFrame as CSV to be used later in Part II.  weatherPy_challenge.csv
   ## I.8. Git hub 
   ## I.9. ?? Using Pandas: How many cities have recorded rainfall or snowfall

